In [1]:
#import libraries
import os
import pandas as pd
import re
import csv
from io import StringIO
from pathlib import Path

In [2]:
#read file
filename = Path(input("Enter the file name : "))

Enter the file name : /Users/gio/Google Drive/Briony share/Financial Records/HSBC Advance Statements/20200115 HSBC Advance Statement 105.csv


In [3]:
#load csv to dataframe
col_list = ['date_raw', 'desc_raw', 'credit_raw', 'debit_raw', 'bal_raw']
df = pd.read_csv(filename, names = col_list)

In [4]:
origdf = df

In [139]:
df = origdf

In [140]:
#clean up date field
new = df["date_raw"].str.split(" ", n = 1, expand = True)

# making separate first name column from new data frame 
df["date"]= new[0] 
  
# making separate last name column from new data frame 
df["desc01"]= new[1] 
  
# Dropping old Name columns 
#df.drop(columns =["Name"], inplace = True)

In [141]:
df = df[['date_raw','date','desc01','desc_raw','credit_raw','debit_raw','bal_raw']]

In [142]:
#fix date
#extract day month year
day = df.date.str.extract(r'(\d{2})', expand = False)
month = df.date.str.extract(r'([A-Za-z]{3})', expand = False)
year = df.date.str.extract(r'(\d{4})', expand = False)
# refill column
df['date'] = pd.to_datetime((month + ' ' + day + ", " + year)).dt.strftime("%m/%d/%Y")


In [143]:
#fix desc
#fix nan
df[['desc01','desc_raw']] = df[['desc01','desc_raw']].fillna('') 
#fix \n
df['desc_raw'] = df.desc_raw.str.replace('\n',' ')
df['desc'] = (df.desc01 + ' ' + df.desc_raw).str.lower()

In [144]:
#drop unneeded columns
df.drop(['date_raw','desc_raw','bal_raw','desc01'], axis = 1, inplace = True)

In [145]:
#fix na of debit and credit rows
df[['credit_raw','debit_raw']] = df[['credit_raw','debit_raw']].fillna(value = 0)

In [146]:
#combine rows that needed to be combined
df['flag'] = df.credit_raw + df.debit_raw
bad_row_list = df[df['flag']==0].index.tolist()
for row in bad_row_list:
    df.iloc[row+1] = df.iloc[row] + df.iloc[row + 1]

df = df.drop(bad_row_list) #get rid of bad rows

#explore later
#df.date.fillna('NaN',inplace=True)
#fund={'date':'first','credit_raw':'sum','debit_raw':'sum','desc':','.join}
#df = df.groupby(df.date.ne(df.date.shift()).cumsum()).agg(fund)

In [147]:
#fill blank dates
df.date = df.date.fillna(method = 'ffill')

In [108]:
#get rid of NaNs
#df = df.fillna(value = 0)

In [148]:
df

,date,credit_raw,debit_raw,desc,flag
0,12/17/2019,0.00,5012.41,lp mka9004v8 hib- 863771153x3dswzi9ph christm...,5012.41
1,12/26/2019,0.00,0.30,tax from ssv ref zdd4-90012,0.30
2,12/26/2019,1.98,0.00,credit interest ref zdd4-90012,1.98
3,01/06/2020,0.00,673.01,pay by 061-032215-130 to 4363 6700 1003 2434 ...,673.01
5,01/06/2020,786.60,0.00,ocmt/u5d788.6 accident health intl underwriti...,786.60


In [153]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/gio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [154]:
from nltk.tokenize import word_tokenize


In [151]:
from nltk.corpus import stopwords

In [155]:
 df['desc_token'] = df.desc.apply(word_tokenize)

In [156]:
df

,date,credit_raw,debit_raw,desc,flag,desc_token
0,12/17/2019,0.00,5012.41,lp mka9004v8 hib- 863771153x3dswzi9ph christm...,5012.41,"[lp, mka9004v8, hib-, 863771153x3dswzi9ph, chr..."
1,12/26/2019,0.00,0.30,tax from ssv ref zdd4-90012,0.30,"[tax, from, ssv, ref, zdd4-90012]"
2,12/26/2019,1.98,0.00,credit interest ref zdd4-90012,1.98,"[credit, interest, ref, zdd4-90012]"
3,01/06/2020,0.00,673.01,pay by 061-032215-130 to 4363 6700 1003 2434 ...,673.01,"[pay, by, 061-032215-130, to, 4363, 6700, 1003..."
5,01/06/2020,786.60,0.00,ocmt/u5d788.6 accident health intl underwriti...,786.60,"[ocmt/u5d788.6, accident, health, intl, underw..."


In [166]:
for sentence in list(df.desc):
    print(word_tokenize(sentence))

['lp', 'mka9004v8', 'hib-', '863771153x3dswzi9ph', 'christmas', 'expenses', 'briony', 'rae', 'eales', '2596232913', 'ayala', 'triangle', 'ref', 'g201-00456']
['tax', 'from', 'ssv', 'ref', 'zdd4-90012']
['credit', 'interest', 'ref', 'zdd4-90012']
['pay', 'by', '061-032215-130', 'to', '4363', '6700', '1003', '2434', 'ref', 'zcpa-00084']
['ocmt/u5d788.6', 'accident', 'health', 'intl', 'underwriting', 'p', 'nofx', 'medical', 'and', 'expenses', 'reimb', 'ursement', 'c0114167', 'r0303050', 'bny', 'cust', 'rrn', '-', 'f1s2001101323100', 'ref', 'yiro-00434']


In [25]:
df.loc[4] = df.loc[4] + df.loc[5]
df

,date,credit_raw,debit_raw,desc
0,12/17/2019,0.00,5012.41,lp mka9004v8 hib- 863771153x3dswzi9ph christm...
1,12/26/2019,0.00,0.30,tax from ssv ref zdd4-90012
2,NaN,1.98,0.00,credit interest ref zdd4-90012
3,01/06/2020,0.00,673.01,pay by 061-032215-130 to 4363 6700 1003 2434 ...
4,NaN,786.60,0.00,ocmt/u5d788.6 accident health intl underwriti...
5,NaN,786.60,0.00,ursement c0114167 r0303050 bny cust rrn - f1s...


In [ ]:
#drop last 3 rows
df.drop(df.tail(3).index,inplace=True)

In [ ]:
#drop unneeded rows
df = df.drop(df[(df.debit_raw.isna() & df.credit_raw.isna())].index)

In [ ]:
#rearrage
df = df[['date','desc','debit_raw','credit_raw']]

In [ ]:
#drop unneeded rows
df = df.loc[-df.debit_raw.str.contains('\n',na=False)]

In [ ]:
#strip unneeded spaces
df.desc = df.desc.str.strip()
df.desc = df.desc.str.replace(" +"," ")

In [ ]:
#export to csv
filename_csv_full = 'for import ' + filename.name
export_csv = df.to_csv (filename_csv_full, index = None, header=True)